In [2]:
from tqdm import tqdm
from requests import request
import pandas as pd
from os import listdir
from os.path import isfile, join

In [3]:
res = request("GET", 'https://www.smard.de/app/chart_data/4359/DE/index_hour.json')
timestamps = res.json()["timestamps"]

In [4]:
time_series = []
for timestamp in tqdm(timestamps):
    res = request("GET", "https://www.smard.de/app/chart_data/4359/DE/4359_DE_hour_" +str(timestamp)+".json")
    body = res.json()["series"]
    time_series += body

100%|██████████| 460/460 [00:32<00:00, 14.35it/s]


In [5]:
residual_load_dataset = pd.DataFrame(time_series)
residual_load_dataset.index = [pd.Timestamp(x, unit='ms') for x in residual_load_dataset.iloc[:, 0]]
residual_load_dataset = residual_load_dataset.drop(columns=0)
residual_load_dataset = residual_load_dataset.dropna()
residual_load_dataset.columns = ["Energy Consumption"]
residual_load_dataset

,Energy Consumption
2014-12-31 23:00:00,35955.75
2015-01-01 00:00:00,34641.00
2015-01-01 01:00:00,32909.25
2015-01-01 02:00:00,31548.00
2015-01-01 03:00:00,30773.50
...,...
2023-10-19 09:00:00,40156.75
2023-10-19 10:00:00,39254.50
2023-10-19 11:00:00,38372.75
2023-10-19 12:00:00,37876.75


In [6]:
def extract_dwd_data(residual_load_dataset, station_id):
    
    station_data = [x for x in listdir() if station_id in x]
    
    wind_data = pd.read_csv([x for x in station_data if "produkt_f" in x][0], sep=";").iloc[-80000:]
    solar_data = pd.read_csv([x for x in station_data if "produkt_sd" in x][0], sep=";").iloc[-80000:]
    temperature_data = pd.read_csv([x for x in station_data if "produkt_tu" in x][0], sep=";").iloc[-80000:]
    precipitation_data = pd.read_csv([x for x in station_data if "produkt_rr" in x][0], sep=";").iloc[-80000:]
    
    wind_data.index = [pd.to_datetime(x, format="%Y%m%d%H") for x in wind_data["MESS_DATUM"]]
    wind_data = wind_data.drop(columns="MESS_DATUM")
    wind_data = wind_data.rename(columns={"  FF": "Wind Velocity for Station " + station_id})
    solar_data.index = [pd.to_datetime(x, format="%Y%m%d%H") for x in solar_data["MESS_DATUM"]]
    solar_data = solar_data.drop(columns="MESS_DATUM")
    solar_data = solar_data.rename(columns={"SD_SO": "Sun Duration for Station " + station_id})
    temperature_data.index = [pd.to_datetime(x, format="%Y%m%d%H") for x in temperature_data["MESS_DATUM"]]
    temperature_data = temperature_data.drop(columns="MESS_DATUM")
    temperature_data = temperature_data.rename(columns={"TT_TU": "Air Temperature for Station " + station_id})
    precipitation_data.index = [pd.to_datetime(x, format="%Y%m%d%H") for x in precipitation_data["MESS_DATUM"]]
    precipitation_data = precipitation_data.drop(columns="MESS_DATUM")
    precipitation_data = precipitation_data.rename(columns={"  R1": "Precipitation Amount for Station " + station_id})

    residual_load_dataset = pd.concat([residual_load_dataset, wind_data["Wind Velocity for Station " + station_id], solar_data["Sun Duration for Station " + station_id], temperature_data["Air Temperature for Station " + station_id], precipitation_data["Precipitation Amount for Station " + station_id]], axis=1)
    
    return residual_load_dataset

In [7]:
for i in ["02014", "03987", "03379", "04928"]:
    residual_load_dataset = extract_dwd_data(residual_load_dataset, i)

In [8]:
residual_load_dataset = residual_load_dataset.dropna()

In [16]:
residual_load_dataset["Quarter"] = residual_load_dataset.index
residual_load_dataset["Quarter"] = residual_load_dataset["Quarter"].dt.quarter

In [18]:
residual_load_dataset["Month"] = [x.month for x in list(residual_load_dataset.index)]

In [24]:
cols = residual_load_dataset.columns.tolist()
cols = cols[1:] + [cols[0]]
residual_load_dataset = residual_load_dataset[cols]
residual_load_dataset 

,Wind Velocity for Station 02014,Sun Duration for Station 02014,Air Temperature for Station 02014,Precipitation Amount for Station 02014,Wind Velocity for Station 03987,Sun Duration for Station 03987,Air Temperature for Station 03987,Precipitation Amount for Station 03987,Wind Velocity for Station 03379,Sun Duration for Station 03379,Air Temperature for Station 03379,Precipitation Amount for Station 03379,Wind Velocity for Station 04928,Sun Duration for Station 04928,Air Temperature for Station 04928,Precipitation Amount for Station 04928,Quarter,Month,Energy Consumption
2015-01-01 03:00:00,3.7,0.0,2.8,0.0,6.0,0.0,3.5,0.0,-999.0,0.0,-3.2,0.0,1.0,0.0,0.5,0.0,1,1,30773.50
2015-01-01 04:00:00,4.2,0.0,2.5,0.0,6.7,0.0,2.6,0.0,-999.0,0.0,-3.3,0.0,1.8,0.0,0.4,0.0,1,1,30040.75
2015-01-01 05:00:00,3.0,0.0,2.0,0.0,5.6,0.0,2.2,0.0,-999.0,0.0,-4.3,0.0,3.2,0.0,0.5,0.0,1,1,28161.50
2015-01-01 06:00:00,3.8,0.0,1.9,0.0,5.7,0.0,2.1,0.0,-999.0,0.0,-4.5,0.0,2.1,0.0,0.6,0.0,1,1,27767.25
2015-01-01 07:00:00,3.1,0.0,1.4,0.0,5.5,0.0,2.7,0.0,-999.0,0.0,-3.9,0.0,0.2,0.0,0.6,0.0,1,1,28531.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 16:00:00,8.6,0.0,16.3,0.0,6.1,0.0,16.3,0.0,1.4,0.0,12.8,0.0,3.8,0.0,17.3,0.0,4,12,15915.75
2022-12-31 17:00:00,7.1,0.0,15.9,0.0,7.7,0.0,16.6,0.0,1.8,0.0,11.4,0.0,3.9,0.0,17.2,0.0,4,12,16438.75
2022-12-31 18:00:00,7.4,0.0,15.7,0.0,8.6,0.0,16.4,0.0,2.5,0.0,11.9,0.0,3.5,0.0,16.6,0.0,4,12,15153.50
2022-12-31 19:00:00,6.7,0.0,16.3,0.0,6.9,0.0,15.8,0.0,2.2,0.0,11.9,0.0,5.1,0.0,15.8,0.0,4,12,11943.75


In [25]:
from sklearn.model_selection import train_test_split

X_test, X_train, y_test, y_train = train_test_split(residual_load_dataset.drop(columns="Energy Consumption"), residual_load_dataset["Energy Consumption"])

In [26]:
from sklearn.ensemble import *
from sklearn.linear_model import *

clf = LinearRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [27]:
import math
from sklearn.metrics import *

math.pow(mean_squared_error(y_test, y_pred), 1/2)

10522.513135778485

In [28]:
test = pd.DataFrame()
test["Y TRUE"] = y_test
test["Y PRED"] = y_pred
test["Difference"] = abs(y_test - y_pred)
test.describe()

,Y TRUE,Y PRED,Difference
count,38867.000000,38867.000000,38867.000000
mean,41620.090726,41459.073054,8631.216598
std,12887.386606,7494.980548,6018.830019
min,2550.000000,-6554.911737,0.348647
25%,33151.625000,36689.370768,3734.521179
50%,42503.000000,42001.883522,7648.243238
75%,50748.375000,46685.807507,12546.588252
max,76049.000000,68916.669866,40392.411737


In [29]:
residual_load_dataset.to_csv("Energy Consumption Dataset.csv")

In [36]:
import numpy as np

y_mean = y_test.mean()
y_pred = np.empty(y_test.size)
y_pred.fill(y_mean)

In [37]:
import math
from sklearn.metrics import *

math.pow(mean_squared_error(y_test, y_pred), 1/2)

12887.220816495656